# Create table Delta Lake

In [2]:
import pyspark
from delta import *

In [3]:
builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

In [4]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/matthew.powers/.ivy2/cache
The jars for the packages stored in: /Users/matthew.powers/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-80c140a0-adbc-49b9-951b-f12bb80a7d02;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.1.0 in central
	found io.delta#delta-storage;2.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 341ms :: artifacts dl 27ms
	:: modules in use:
	io.delta#delta-core_2.12;2.1.0 from central in [default]
	io.delta#delta-storage;2.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number|

22/09/16 13:29:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Create Table for Delta Lake from PySpark DataFrame

In [4]:
spark.sql("DROP TABLE IF EXISTS table1").show()

++
||
++
++



In [5]:
columns = ["character", "franchise"]
data = [("link", "zelda"), ("king k rool", "donkey kong"), ("samus", "metroid")]
rdd = spark.sparkContext.parallelize(data)
df = rdd.toDF(columns)

In [6]:
df.show()

+-----------+-----------+
|  character|  franchise|
+-----------+-----------+
|       link|      zelda|
|king k rool|donkey kong|
|      samus|    metroid|
+-----------+-----------+



In [7]:
df.write.format("delta").saveAsTable("table1")

In [8]:
spark.sql("SELECT * FROM table1").show()

+-----------+-----------+
|  character|  franchise|
+-----------+-----------+
|king k rool|donkey kong|
|      samus|    metroid|
|       link|      zelda|
+-----------+-----------+



In [9]:
DeltaTable.isDeltaTable(spark, "spark-warehouse/table1")

True

In [10]:
df.write.format("parquet").saveAsTable("table1_as_parquet")

In [11]:
spark.sql("SELECT * FROM table1_as_parquet").show()

+-----------+-----------+
|  character|  franchise|
+-----------+-----------+
|king k rool|donkey kong|
|      samus|    metroid|
|       link|      zelda|
+-----------+-----------+

22/09/12 10:25:40 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /private/var/folders/19/_52w4zps3xjc6plz_f63j8sh0000gp/T/spark-ee858e6d-5653-4d42-afde-4e1f97db4480/pyspark-fc267009-acdf-45d7-8cfb-44594c43afb2. Falling back to Java IO way
java.io.IOException: Failed to delete: /private/var/folders/19/_52w4zps3xjc6plz_f63j8sh0000gp/T/spark-ee858e6d-5653-4d42-afde-4e1f97db4480/pyspark-fc267009-acdf-45d7-8cfb-44594c43afb2
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:171)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:110)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:91)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1206)
	at org.apache.spark.util.Shutd

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 52942)
Traceback (most recent call last):
  File "/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/socketserver.py", line 747, in __init__
    self.handle()
  File "/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/site-packages/pyspark/accumulators.py", line 281, in ha

## Create Delta Lake table with SQL

In [16]:
spark.sql("""
  CREATE TABLE table2 (country STRING, continent STRING) USING delta
""")

DataFrame[]

In [17]:
spark.sql("""
  INSERT INTO table2 VALUES
      ('china', 'asia'),
      ('argentina', 'south america')
""")

DataFrame[]

In [18]:
spark.sql("SELECT * FROM table2").show()

+---------+-------------+
|  country|    continent|
+---------+-------------+
|argentina|south america|
|    china|         asia|
+---------+-------------+



In [19]:
DeltaTable.isDeltaTable(spark, "spark-warehouse/table2")

True

## Create table with Delta Lake Python API

In [1]:
from pyspark.sql.types import *

In [7]:
dt1 = (DeltaTable.create(spark)
    .tableName("testTable1")
    .addColumn("c1", dataType = "INT", nullable = False)
    .addColumn("c2", dataType = IntegerType(), generatedAlwaysAs = "c1 + 1")
    .partitionedBy("c1")
    .execute())

In [8]:
dt1

In [12]:
spark.sql("DESCRIBE TABLE testTable1").show()

+--------------+---------+-------+
|      col_name|data_type|comment|
+--------------+---------+-------+
|            c1|      int|       |
|            c2|      int|       |
|              |         |       |
|# Partitioning|         |       |
|        Part 0|       c1|       |
+--------------+---------+-------+



In [10]:
dt2 = (DeltaTable.createIfNotExists(spark)
    .tableName("testTable2")
    .addColumn("c1", dataType = "INT", nullable = False)
    .addColumn("c2", dataType = IntegerType(), generatedAlwaysAs = "c1 + 1")
    .partitionedBy("c1")
    .execute())

In [11]:
dt2

## Create Delta Lake table from CSV

In [23]:
df = spark.read.option("header", True).csv("../../data/students/students1.csv")

In [24]:
df.show()

+------------+---------------+-------+
|student_name|graduation_year|  major|
+------------+---------------+-------+
|someXXperson|           2023|   math|
|     liXXyao|           2025|physics|
+------------+---------------+-------+



In [25]:
df.write.format("delta").saveAsTable("students")

In [26]:
spark.sql("SELECT * from students").show()

+------------+---------------+-------+
|student_name|graduation_year|  major|
+------------+---------------+-------+
|someXXperson|           2023|   math|
|     liXXyao|           2025|physics|
+------------+---------------+-------+



In [27]:
DeltaTable.isDeltaTable(spark, "spark-warehouse/students")

True

## Cleanup

In [11]:
!rm -rf spark-warehouse